# Getting started with Interactive Brokers

Our project connects to Interactive Brokers to perform trades.

Interactive Brokers has two ways of accessing the API:
* **TWS** - this is the main desktop client. It's useful for seeing what's going on and testing things. Shuts down automatically after 24 hours.
* **Gateway** - this is a lightweight API client. It's still java based and requires a GUI login (there is no 'headless' API), but it's suitable for running a live system.

There are also three levels of login:
* Demo
* Simulated/paper trading
* Live trading

These environments can quite different in behaviour (in terms of what instruments are available, and especially what historical/live data is available).

## Using the demo Interactive Brokers mode to test API connectivity

Download and install the latest TWS onto your computer.

Open it and log in with:
* Username: edemo
* Password: demouser

Go into the settings and configure the API:
* Enable the API on port 4003
* Disable Read-only API

Our system has a module at core.ibstate which handles the connection to IB. IB's API is realtime & event driven.

The following code should connect to IB:

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import os
os.chdir('..')
import pandas as pd
import numpy as np
import trading.start
import trading.portfolio as portfolio
import config.settings
from time import sleep
from core.utility import *
from trading.accountcurve import *
import data.db_mongo as db
import config.portfolios
from pylab import rcParams
rcParams['figure.figsize'] = 15, 10
p = portfolio.Portfolio(instruments=config.portfolios.p_trade)
i = p.instruments
from trading.ibstate import *

ib = IBstate()

IB has recently introduced a Python API, but we don't use it yet (we currently use IbPy, which is a Python module that maps to the Java API https://github.com/blampe/IbPy). 

The official API docs are here:
http://interactivebrokers.github.io/tws-api/

The API consists of two main components:
* Client
* Wrapper

We send instructions to the API with the Client, and receive responses back via Wrapper. We have to subscribe to various event streams to receive responses from the API. IBstate tries to collect all these realtime responses to maintain a 'stateful' respresentation of what is going on. For example, our account balance ('net liquidiation') will change throughout the day. IBstate listens for these updates and remembers them, so when we call:

In [ ]:
ib.net()

it returns a result from our local buffer.

IBstate can also collect responses and join them together (for example, portfolios are returned line by line, IBstate joins them together into a Pandas dataframe, so we can work it more easily).

IBState also allows us to trade. The magic command is:

In [ ]:
ib.sync_portfolio(p, trade = True)

When you run this, you should start to see the trades go into TWS. It pauses for 60 seconds between each trade, so this will take several minutes to run.

If you refresh the following command, you should see a mirror image of what you see in the TWS interface.

In [ ]:
ib.portfolio